In [ ]:
## preprocessing

## libraries
import glob
import random
import pretty_midi
import IPython
import numpy as np
from tqdm import tnrange, tqdm
from tqdm import notebook as tqdm
from random import shuffle, seed
import numpy as np

import unicodedata
import re
import numpy as np
import os
import io
import time

## how to unzip a folder of midi files in the notebook before running preprocessing
#unzips the folder
import zipfile
with zipfile.ZipFile("arabic_tribal_rhythms_ZIP.zip","r") as zip_ref:
    zip_ref.extractall()#where you want the file to be unzipped, empty means I want it in the same folder?


In [12]:
#1 

#i want this to take a folder of midi files and produce a list
class preprocessing_class:
    
    
    def __init__(self, folder, batch_size, seq_len):
        self.folder = folder
        self.batch_size = batch_size
        self.seq_len = seq_len

#1
    def get_list_midi(folder = self.folder, seed_int = 666):
        list_all_midi = glob.glob(self.folder)
        seed(seed_int)
        shuffle(list_all_midi)
        return list_all_midi
        
    """Get the list of all midi file in the folders

      Parameters
      ==========
      folder : str
        The midi folder.
      seed_int : int
        the random seed.

      Returns
      =======
      The midi files
      """
    def generate_batch_song(list_all_midi,  batch_size = self.batch_size, start_index=0, fs=30, seq_len=self.seq_len, use_tqdm=False):
    """
    Generate Batch music that will be used to be input and output of the neural network
    
    Parameters
    ==========
    list_all_midi : list
      List of midi files
    batch_size : int
      A number of music in one batch
    start_index : int
      The start index to be batched in list_all_midi
    fs : int
      Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    seq_len : int
      The sequence length of the music to be input of neural network
    use_tqdm : bool
      Whether to use tqdm or not in the function
    
    Returns
    =======
    Tuple of input and target neural network
    
    """
    
        assert len(list_all_midi) >= batch_size
        dict_time_notes = generate_dict_time_notes(list_all_midi, batch_size, start_index, fs, use_tqdm=use_tqdm)

        list_musics = process_notes_in_song(dict_time_notes, seq_len)
        collected_list_input, collected_list_target = [], []

        for music in list_musics:
            list_training, list_target = generate_input_and_target(music, seq_len)
            collected_list_input += list_training
            collected_list_target += list_target
        return collected_list_input, collected_list_target


    def generate_dict_time_notes(list_all_midi, batch_size = self.batch_size, start_index=0, fs=30, use_tqdm=True):
        """ Generate map (dictionary) of music ( in index ) to piano_roll (in np.array)

        Parameters
        ==========
        list_all_midi : list
            List of midi files
        batch_size : int
          A number of music in one batch
        start_index : int
          The start index to be batched in list_all_midi
        fs : int
          Sampling frequency of the columns, i.e. each column is spaced apart
            by ``1./fs`` seconds.
        use_tqdm : bool
          Whether to use tqdm or not in the function

        Returns
        =======
        dictionary of music to piano_roll (in np.array)"""
        assert len(list_all_midi) >= batch_size
        dict_time_notes = {}
        process_tqdm_midi = tqdm.tqdm(range(start_index, min(start_index + batch_size, len(list_all_midi)))) if use_tqdm else range(start_index,  min(start_index + batch_size, len(list_all_midi)))
        for i in process_tqdm_midi:
            midi_file_name = list_all_midi[i]
            if use_tqdm:
                process_tqdm_midi.set_description("Processing {}".format(midi_file_name))
            try: # Handle exception on malformat MIDI files
                midi_pretty_format = pretty_midi.PrettyMIDI(midi_file_name)
                piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
                piano_roll = piano_midi.get_piano_roll(fs=fs)
                dict_time_notes[i] = piano_roll
            except Exception as e:
                print(e)
                print("broken file : {}".format(midi_file_name))
                pass
        return dict_time_notes
    
    def preprocess_midi(self.folder):
        list_all_midi = get_list_midi()
        dictionary = generate_dict_time_notes(list_all_midi, batch_size = 16, start_index=0, fs=30, use_tqdm=True)
        return dictionary 


IndentationError: expected an indented block (<ipython-input-12-63dde01df9dd>, line 56)